In [5]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [57]:
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    if isinstance(url,str):
        response = requests.get(url)
    else:
        #if already it is a loaded html page
        response = url
    if response is not None:
        html = BeautifulSoup(response.text, 'html.parser')
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
        if search:
            soup = html            
            r = ''
            if 'find' in search.keys():
                print('finding',search['find'])
                soup = soup.find(**search['find'])
                r = soup
            if 'find_all' in search.keys():
                print('finding all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
        
        return r

In [70]:
#getting the africa leaders
url = "https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/"
res_gov = get_elements(url, search={'find_all':{'class_':'wp-block-embed__wrapper'}}) 
res_gv = get_elements(url, search={'find_all':{'class_':'twitter-tweet'}})
officials = []
for i in range(2 ,len(res_gv)):
    mainblk = res_gv[i]
    olist = mainblk.find_all(text=True , recursive=False)
    officials.append(olist)
final = []
for item in officials:
    final.append(item[0])

finallist = pd.DataFrame(final, columns=['User'])
top10 = finallist.head(10)
top10.to_csv('africaleaders.csv',index=True )

finding all of {'class_': 'wp-block-embed__wrapper'}
finding all of {'class_': 'twitter-tweet'}


In [85]:
#getting africa freak data 
def scrape(url):
	r = requests.get(url)
	soup = BeautifulSoup(r.content , features="html.parser")
	return soup

url = "https://africafreak.com/100-most-influential-twitter-users-in-africa"
influencers = scrape(url)
names = influencers.findAll("h2")
fulllist = []
for name in names:
    thisname = name.text
    namenno = thisname.split(".")
    fulllist.append(namenno)
influencerrank = pd.DataFrame(fullist , columns=['Rank', 'User'])
highest10 = influencerrank.tail(10)
highest10.to_csv('top_influencers.csv', index=False)

[['100', ' Jeffrey Gettleman (@gettleman)'],
 ['99', ' Africa24 Media (@a24media)'],
 ['98', ' Scapegoat (@andiMakinana)'],
 ['97', ' Africa Check (@AfricaCheck)'],
 ['96', ' James Copnall (@JamesCopnall)'],
 ['95', ' Online Africa (@oafrica)'],
 ['94', ' Patrick Ngowi (@PatrickNgowi)'],
 ['93', ' DOS African Affairs (@StateAfrica)'],
 ['92', ' MoadowAJE (@Moadow)'],
 ['91', ' Brendan Boyle (@BrendanSAfrica)'],
 ['90', ' City of Tshwane (@CityTshwane)'],
 ['89', ' VISI Magazine (@VISI_Mag)'],
 ['88', ' andBeyond (@andBeyondSafari)'],
 ['87', ' This Is Africa (@ThisIsAfricaTIA)'],
 ['86', ' Sarah Carter (@sarzss)'],
 ['85', ' The EIU Africa team (@TheEIU_Africa)'],
 ['84', ' Investing In Africa (@InvestInAfrica)'],
 ['83', ' Barry Malone (@malonebarry)'],
 ['82', ' ARTsouthAFRICA (@artsouthafrica)'],
 ['81', ' Kahn Morbee (@KahnMorbee)'],
 ['80', ' Jamal Osman (@JamalMOsman)'],
 ['79', ' iamsuede™ (@iamsuede)'],
 ['78', ' Mike Stopforth (@mikestopforth)'],
 ['77', ' Equal Education (@eq